In [1]:
'''
This file plots CMIP5 RCP - HIST over South Australia
/Users/earl/Desktop/Yang
and places the outputs in
/Users/earl/Dropbox/CMIP5/figures

Earl Duran 
created: 19-Mar-18
e.duran@unsw.edu.au
'''

import cosima_cookbook as cc
import os
import xarray as xr
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib
import sys
import pickle
import itertools
def find_nearest_index(array, value):
    return int((np.abs(array - value)).argmin())
from scipy import interpolate
from scipy import stats
from dask.distributed import Client

import cartopy.crs as ccrs
import cartopy.feature as cft
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
def arange(start,end,stride=1):
    return np.arange(start,end+0.00000001,stride)

import sys, os
sys.path.append(os.path.join(os.getcwd(), '..'))  # so we can import ../exptdata
import exptdata

session = cc.database.create_session()

In [2]:
%%javascript
IPython.notebook.kernel.execute('nb_name = ' + '"' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [3]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:40026 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 33.67 GB


In [4]:
input_path = '/g/data/e14/erd561/IMOS/'

output_path = '/g/data/e14/erd561/Australia_3/'

In [8]:
year = np.arange(1993,2018)

lonW = 88
lonE = 182

latS = -72
latN = 22

sea_level_z0_month = xr.open_dataset(input_path + 'IMOS_aggregation_AVISO_new_20190917T035303Z.nc').\
                                GSL.\
                sel(LONGITUDE=slice(lonW,lonE)).\
                sel(LATITUDE=slice(latS,latN))*100
print(sea_level_z0_month)

sea_level_z0 = sea_level_z0_month.\
                groupby('TIME.year').\
                mean('TIME')
print(sea_level_z0)

sea_level_z0 = sea_level_z0.sel(year=slice(1993,2017))
print(sea_level_z0)

<xarray.DataArray 'GSL' (TIME: 5834, LATITUDE: 351, LONGITUDE: 451)>
array([[[ 13.82509575,  11.76506493,   9.79659103, ...,  82.66064426,
          83.78984634,  85.37683305],
        [  9.64399615,   7.33981353,   5.23400424, ...,  83.75932736,
          85.62098485,  87.84887003],
        [  4.4252514 ,   2.4567775 ,   0.50356309, ...,  85.22423817,
          87.6504967 ,  90.36668549],
        ...,
        [238.5973479 , 239.11617048, 239.86388537, ..., 215.31136984,
         215.26559138, 214.79254727],
        [238.41423405, 238.79572124, 239.46713869, ..., 215.15877497,
         215.23507241, 214.88410419],
        [238.42949354, 238.56682893, 239.07039202, ..., 215.25033189,
         215.46396472, 215.29611036]],

       [[  7.95019303,   6.69891505,   5.58497246, ...,  83.53043505,
          84.2018525 ,  84.85801047],
        [  3.47916317,   2.3347016 ,   1.17498054, ...,  84.29340943,
          85.48364946,  86.59759205],
        [ -1.63276517,  -2.27366365,  -3.25027086, .

In [ ]:
lat = sea_level_z0.LATITUDE
lon = sea_level_z0.LONGITUDE

# sea_level_z0_trans = sea_level_z0.transpose('LATITUDE', 'LONGITUDE', 'year')
sea_level_z0_trans = sea_level_z0
print(np.shape(sea_level_z0_trans))
sea_level_z0_slope = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='sea_level_z0')
print(np.shape(sea_level_z0_slope))
sea_level_z0_p_value = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='sea_level_z0')
sea_level_z0_std_err = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='sea_level_z0')
for iid, i in enumerate(lat):
    for jid, j in enumerate(lon):
        sea_level_z0_slope[iid, jid], _, _, sea_level_z0_p_value[iid, jid], sea_level_z0_std_err[iid, jid] = \
        stats.linregress(year, sea_level_z0_trans[:, iid, jid])
    print('lat ' + str(np.array(i)))
print(sea_level_z0_slope)

(25, 351, 451)
(351, 451)
lat -60.0
lat -59.8
lat -59.6
lat -59.4
lat -59.2
lat -59.0
lat -58.8
lat -58.6
lat -58.4
lat -58.2
lat -58.0
lat -57.8
lat -57.6
lat -57.4
lat -57.2
lat -57.0
lat -56.8
lat -56.6
lat -56.4
lat -56.2
lat -56.0
lat -55.8
lat -55.6
lat -55.4
lat -55.2
lat -55.0
lat -54.8
lat -54.6
lat -54.4
lat -54.2
lat -54.0
lat -53.8
lat -53.6
lat -53.4
lat -53.2
lat -53.0
lat -52.8
lat -52.6
lat -52.4
lat -52.2
lat -52.0
lat -51.8
lat -51.6
lat -51.4
lat -51.2
lat -51.0
lat -50.8
lat -50.6
lat -50.4
lat -50.2
lat -50.0
lat -49.8
lat -49.6
lat -49.4
lat -49.2
lat -49.0
lat -48.8
lat -48.6
lat -48.4
lat -48.2
lat -48.0
lat -47.8
lat -47.6
lat -47.4
lat -47.2
lat -47.0
lat -46.8
lat -46.6
lat -46.4
lat -46.2
lat -46.0
lat -45.8
lat -45.6
lat -45.4
lat -45.2
lat -45.0
lat -44.8
lat -44.6
lat -44.4
lat -44.2
lat -44.0
lat -43.8
lat -43.6
lat -43.4
lat -43.2
lat -43.0
lat -42.8
lat -42.6
lat -42.4
lat -42.2
lat -42.0
lat -41.8
lat -41.6
lat -41.4
lat -41.2
lat -41.0
lat -40.8
lat 

In [ ]:
# literally copy-pasted from 
# /g/data/hh5/tmp/cosima/access-om2/1deg_jra55v13_iaf_spinup1_B1/output059/ocean

lonW = 90
lonE = 180

latS = -70
latN = 20

lat_1 = xr.open_dataset(
'/g/data/hh5/tmp/cosima/access-om2/1deg_jra55v13_iaf_spinup1_B1/output059/ocean/ocean.nc').\
yt_ocean.sel(yt_ocean=slice(latS,latN)).values

lon_1 = xr.open_dataset(
'/g/data/hh5/tmp/cosima/access-om2/1deg_jra55v13_iaf_spinup1_B1/output059/ocean/ocean.nc').\
xt_ocean.sel(xt_ocean=slice(lonW-360,lonE-360)).values+360

grid_lon_1, grid_lat_1 = np.meshgrid(lon_1,lat_1)

sea_level_z0_grid_lon, sea_level_z0_grid_lat = np.meshgrid(sea_level_z0.LONGITUDE, sea_level_z0.LATITUDE)
sea_level_z0_lon_flat = np.transpose(sea_level_z0_grid_lon.flatten('F'))
sea_level_z0_lat_flat = sea_level_z0_grid_lat.flatten('F')
sea_level_z0_flat_list = [sea_level_z0_lon_flat, sea_level_z0_lat_flat]    
sea_level_z0_grid_points = np.transpose(sea_level_z0_flat_list)

for y in year:
    sea_level_z0_flatten = sea_level_z0.sel(year=y).stack(z=('LONGITUDE', 'LATITUDE'))

    sea_level_z0_1_now = interpolate.griddata(sea_level_z0_grid_points, sea_level_z0_flatten, 
                                 (grid_lon_1, grid_lat_1), 
                                 method='linear')
    
    if y == 1993:
        sea_level_z0_1 = sea_level_z0_1_now
    else:
        sea_level_z0_1 = np.dstack((sea_level_z0_1, sea_level_z0_1_now))
        
    print(y)

print(np.shape(sea_level_z0_1))

In [ ]:
sea_level_z0_1_trans = np.transpose(sea_level_z0_1, (2,1,0))
print(np.shape(sea_level_z0_1_trans))
sea_level_z0_1_slope = xr.DataArray(
    np.zeros([np.shape(lat_1)[0], np.shape(lon_1)[0]]), dims=(
    'lat_1', 'lon_1'), coords=[lat_1, lon_1], name='sea_level_z0_1')
sea_level_z0_1_p_value = xr.DataArray(
    np.zeros([np.shape(lat_1)[0], np.shape(lon_1)[0]]), dims=(
    'lat_1', 'lon_1'), coords=[lat_1, lon_1], name='sea_level_z0')
sea_level_z0_1_std_err = xr.DataArray(
    np.zeros([np.shape(lat_1)[0], np.shape(lon_1)[0]]), dims=(
    'lat_1', 'lon_1'), coords=[lat_1, lon_1], name='sea_level_z0')
for iid, i in enumerate(lat_1):
    for jid, j in enumerate(lon_1):
        sea_level_z0_1_slope[iid, jid], _, _, sea_level_z0_1_p_value[iid, jid], sea_level_z0_1_std_err[iid, jid] = \
        stats.linregress(year, sea_level_z0_1_trans[:, jid, iid])
    print('lat_1 ' + str(np.array(i)))
print(sea_level_z0_1_slope)

In [ ]:
# literally copy-pasted from 
# /g/data/hh5/tmp/cosima/access-om2-025/025deg_jra55v13_iaf_gmredi6/output153/ocean

lat_025 = xr.open_dataset(
'/g/data/hh5/tmp/cosima/access-om2-025/025deg_jra55v13_iaf_gmredi6/output153/ocean/ocean.nc').\
yt_ocean.sel(yt_ocean=slice(latS,latN)).values

lon_025 = xr.open_dataset(
'/g/data/hh5/tmp/cosima/access-om2-025/025deg_jra55v13_iaf_gmredi6/output153/ocean/ocean.nc').\
xt_ocean.sel(xt_ocean=slice(lonW-360,lonE-360)).values+360

grid_lon_025, grid_lat_025 = np.meshgrid(lon_025,lat_025)

sea_level_z0_grid_lon, sea_level_z0_grid_lat = np.meshgrid(sea_level_z0.LONGITUDE, sea_level_z0.LATITUDE)
sea_level_z0_lon_flat = np.transpose(sea_level_z0_grid_lon.flatten('F'))
sea_level_z0_lat_flat = sea_level_z0_grid_lat.flatten('F')
sea_level_z0_flat_list = [sea_level_z0_lon_flat, sea_level_z0_lat_flat]    
sea_level_z0_grid_points = np.transpose(sea_level_z0_flat_list)

for y in year:
    sea_level_z0_flatten = sea_level_z0.sel(year=y).stack(z=('LONGITUDE', 'LATITUDE'))

    sea_level_z0_025_now = interpolate.griddata(sea_level_z0_grid_points, sea_level_z0_flatten, 
                                 (grid_lon_025, grid_lat_025), 
                                 method='linear')
    
    if y == 1993:
        sea_level_z0_025 = sea_level_z0_025_now
    else:
        sea_level_z0_025 = np.dstack((sea_level_z0_025, sea_level_z0_025_now))
        
    print(y)

print(np.shape(sea_level_z0_025))

In [ ]:
sea_level_z0_025_trans = np.transpose(sea_level_z0_025, (2,1,0))
print(np.shape(sea_level_z0_025_trans))
sea_level_z0_025_slope = xr.DataArray(
    np.zeros([np.shape(lat_025)[0], np.shape(lon_025)[0]]), dims=(
    'lat_025', 'lon_025'), coords=[lat_025, lon_025], name='sea_level_z0_025')
sea_level_z0_025_p_value = xr.DataArray(
    np.zeros([np.shape(lat_025)[0], np.shape(lon_025)[0]]), dims=(
    'lat_025', 'lon_025'), coords=[lat_025, lon_025], name='sea_level_z0')
sea_level_z0_025_std_err = xr.DataArray(
    np.zeros([np.shape(lat_025)[0], np.shape(lon_025)[0]]), dims=(
    'lat_025', 'lon_025'), coords=[lat_025, lon_025], name='sea_level_z0')
for iid, i in enumerate(lat_025):
    for jid, j in enumerate(lon_025):
        sea_level_z0_025_slope[iid, jid], _, _, sea_level_z0_025_p_value[iid, jid], sea_level_z0_025_std_err[iid, jid] = \
        stats.linregress(year, sea_level_z0_025_trans[:, jid, iid])
    print('lat_025 ' + str(np.array(i)))
print(sea_level_z0_025_slope)

In [ ]:
# literally copy-pasted from 
# /g/data/hh5/tmp/cosima/access-om2-01/01deg_jra55v13_iaf/output197/ocean/ocean.nc

lat_01 = xr.open_dataset(
'/g/data/hh5/tmp/cosima/access-om2-01/01deg_jra55v13_iaf/output197/ocean/ocean.nc').\
yt_ocean.sel(yt_ocean=slice(latS,latN)).values

lon_01 = xr.open_dataset(
'/g/data/hh5/tmp/cosima/access-om2-01/01deg_jra55v13_iaf/output197/ocean/ocean.nc').\
xt_ocean.sel(xt_ocean=slice(lonW-360,lonE-360)).values+360

grid_lon_01, grid_lat_01 = np.meshgrid(lon_01,lat_01)

sea_level_z0_grid_lon, sea_level_z0_grid_lat = np.meshgrid(sea_level_z0.LONGITUDE, sea_level_z0.LATITUDE)
sea_level_z0_lon_flat = np.transpose(sea_level_z0_grid_lon.flatten('F'))
sea_level_z0_lat_flat = sea_level_z0_grid_lat.flatten('F')
sea_level_z0_flat_list = [sea_level_z0_lon_flat, sea_level_z0_lat_flat]    
sea_level_z0_grid_points = np.transpose(sea_level_z0_flat_list)

for y in year:
    sea_level_z0_flatten = sea_level_z0.sel(year=y).stack(z=('LONGITUDE', 'LATITUDE'))

    sea_level_z0_01_now = interpolate.griddata(sea_level_z0_grid_points, sea_level_z0_flatten, 
                                 (grid_lon_01, grid_lat_01), 
                                 method='linear')
    
    if y == 1993:
        sea_level_z0_01 = sea_level_z0_01_now
    else:
        sea_level_z0_01 = np.dstack((sea_level_z0_01, sea_level_z0_01_now))
        
    print(y)

print(np.shape(sea_level_z0_01))

In [ ]:
sea_level_z0_01_trans = np.transpose(sea_level_z0_01, (2,1,0))
print(np.shape(sea_level_z0_01_trans))
sea_level_z0_01_slope = xr.DataArray(
    np.zeros([np.shape(lat_01)[0], np.shape(lon_01)[0]]), dims=(
    'lat_01', 'lon_01'), coords=[lat_01, lon_01], name='sea_level_z0_01')
sea_level_z0_01_p_value = xr.DataArray(
    np.zeros([np.shape(lat_01)[0], np.shape(lon_01)[0]]), dims=(
    'lat_01', 'lon_01'), coords=[lat_01, lon_01], name='sea_level_z0')
sea_level_z0_01_std_err = xr.DataArray(
    np.zeros([np.shape(lat_01)[0], np.shape(lon_01)[0]]), dims=(
    'lat_01', 'lon_01'), coords=[lat_01, lon_01], name='sea_level_z0')
for iid, i in enumerate(lat_01):
    for jid, j in enumerate(lon_01):
        sea_level_z0_01_slope[iid, jid], _, _, sea_level_z0_01_p_value[iid, jid], sea_level_z0_01_std_err[iid, jid] = \
        stats.linregress(year, sea_level_z0_01_trans[:, jid, iid])
    print('lat_01 ' + str(np.array(i)))
print(sea_level_z0_01_slope)

In [ ]:
sea_level_z0_xr = xr.DataArray(np.transpose(sea_level_z0.values, (1,2,0)), name='sea_level_z0_obs',
                        coords=[lat, lon, year], 
                        dims=['lat', 'lon', 'year'])
print(sea_level_z0_xr)
sea_level_z0_xr.to_netcdf(output_path + 'sea_level_z0_obs.nc')

sea_level_z0_slope_xr = xr.DataArray(sea_level_z0_slope, name='sea_level_z0_slope_obs',
                        coords=[lat, lon], 
                        dims=['lat', 'lon'])
print(sea_level_z0_slope_xr)
sea_level_z0_slope_xr.to_netcdf(output_path + 'sea_level_z0_slope_obs.nc')

sea_level_z0_p_value_xr = xr.DataArray(sea_level_z0_p_value, name='sea_level_z0_p_value_obs',
                        coords=[lat, lon], 
                        dims=['lat', 'lon'])
print(sea_level_z0_p_value_xr)
sea_level_z0_p_value_xr.to_netcdf(output_path + 'sea_level_z0_p_value_obs.nc')

sea_level_z0_1_p_value_xr = xr.DataArray(sea_level_z0_1_p_value, name='sea_level_z0_1_p_value_obs',
                        coords=[lat_1, lon_1], 
                        dims=['lat_1', 'lon_1'])
print(sea_level_z0_1_p_value_xr)
sea_level_z0_1_p_value_xr.to_netcdf(output_path + 'sea_level_z0_1_p_value_obs.nc')

sea_level_z0_std_err_xr = xr.DataArray(sea_level_z0_std_err, name='sea_level_z0_std_err_obs',
                        coords=[lat, lon], 
                        dims=['lat', 'lon'])
print(sea_level_z0_std_err_xr)
sea_level_z0_std_err_xr.to_netcdf(output_path + 'sea_level_z0_std_err_obs.nc')




sea_level_z0_1_xr = xr.DataArray(sea_level_z0_1, name='sea_level_z0_1_obs',
                        coords=[lat_1, lon_1, year], 
                        dims=['lat_1', 'lon_1', 'year'])
print(sea_level_z0_1_xr)
sea_level_z0_1_xr.to_netcdf(output_path + 'sea_level_z0_1_obs.nc')

sea_level_z0_1_slope_xr = xr.DataArray(sea_level_z0_1_slope, name='sea_level_z0_1_slope_obs',
                        coords=[lat_1, lon_1], 
                        dims=['lat_1', 'lon_1'])
print(sea_level_z0_1_slope_xr)
sea_level_z0_1_slope_xr.to_netcdf(output_path + 'sea_level_z0_1_slope_obs.nc')

sea_level_z0_1_std_err_xr = xr.DataArray(sea_level_z0_1_std_err, name='sea_level_z0_1_std_err_obs',
                        coords=[lat_1, lon_1], 
                        dims=['lat_1', 'lon_1'])
print(sea_level_z0_1_std_err_xr)
sea_level_z0_1_std_err_xr.to_netcdf(output_path + 'sea_level_z0_1_std_err_obs.nc')




sea_level_z0_025_xr = xr.DataArray(sea_level_z0_025, name='sea_level_z0_025_obs',
                        coords=[lat_025, lon_025, year], 
                        dims=['lat_025', 'lon_025', 'year'])
print(sea_level_z0_025_xr)
sea_level_z0_025_xr.to_netcdf(output_path + 'sea_level_z0_025_obs.nc')

sea_level_z0_025_slope_xr = xr.DataArray(sea_level_z0_025_slope, name='sea_level_z0_025_slope_obs',
                        coords=[lat_025, lon_025], 
                        dims=['lat_025', 'lon_025'])
print(sea_level_z0_025_slope_xr)
sea_level_z0_025_slope_xr.to_netcdf(output_path + 'sea_level_z0_025_slope_obs.nc')

sea_level_z0_025_p_value_xr = xr.DataArray(sea_level_z0_025_p_value, name='sea_level_z0_025_p_value_obs',
                        coords=[lat_025, lon_025], 
                        dims=['lat_025', 'lon_025'])
print(sea_level_z0_025_p_value_xr)
sea_level_z0_025_p_value_xr.to_netcdf(output_path + 'sea_level_z0_025_p_value_obs.nc')

sea_level_z0_025_std_err_xr = xr.DataArray(sea_level_z0_025_std_err, name='sea_level_z0_025_std_err_obs',
                        coords=[lat_025, lon_025], 
                        dims=['lat_025', 'lon_025'])
print(sea_level_z0_025_std_err_xr)
sea_level_z0_025_std_err_xr.to_netcdf(output_path + 'sea_level_z0_025_std_err_obs.nc')




sea_level_z0_01_xr = xr.DataArray(sea_level_z0_01, name='sea_level_z0_01_obs',
                        coords=[lat_01, lon_01, year], 
                        dims=['lat_01', 'lon_01', 'year'])
print(sea_level_z0_01_xr)
sea_level_z0_01_xr.to_netcdf(output_path + 'sea_level_z0_01_obs.nc')

sea_level_z0_01_slope_xr = xr.DataArray(sea_level_z0_01_slope, name='sea_level_z0_01_slope_obs',
                        coords=[lat_01, lon_01], 
                        dims=['lat_01', 'lon_01'])
print(sea_level_z0_01_slope_xr)
sea_level_z0_01_slope_xr.to_netcdf(output_path + 'sea_level_z0_01_slope_obs.nc')

sea_level_z0_01_p_value_xr = xr.DataArray(sea_level_z0_01_p_value, name='sea_level_z0_01_p_value_obs',
                        coords=[lat_01, lon_01], 
                        dims=['lat_01', 'lon_01'])
print(sea_level_z0_01_p_value_xr)
sea_level_z0_01_p_value_xr.to_netcdf(output_path + 'sea_level_z0_01_p_value_obs.nc')

sea_level_z0_01_std_err_xr = xr.DataArray(sea_level_z0_01_std_err, name='sea_level_z0_01_std_err_obs',
                        coords=[lat_01, lon_01], 
                        dims=['lat_01', 'lon_01'])
print(sea_level_z0_01_std_err_xr)
sea_level_z0_01_std_err_xr.to_netcdf(output_path + 'sea_level_z0_01_std_err_obs.nc')